# 02. 트래픽 관리 (Traffic Management)

## 학습 목표

이 노트북에서는 Istio의 **트래픽 관리** 기능을 학습합니다.

**테스트 시나리오:**
1. Canary 배포 (가중치 기반 라우팅)
2. 헤더 기반 라우팅

**핵심 개념:**
- **VirtualService**: 트래픽 분배 규칙 정의
- **DestinationRule**: 서비스 버전(subset) 정의

**학습 포인트:**
- 코드 수정 없이 배포 전략 구현
- 점진적 롤아웃으로 위험 최소화
- A/B 테스트 구현

---

## 사전 조건 체크

In [ ]:
# 환경 확인
!kubectl get pods -n istio-demo | grep -E "httpbin|sleep"

---

## 개념 설명: 트래픽 관리

### 전통적인 배포 vs Service Mesh

**전통적인 Kubernetes 배포:**
- Deployment replicas 조정으로 트래픽 분배
- 세밀한 제어 불가능
- 롤백 시 전체 재배포 필요

**Istio 트래픽 관리:**
- 정확한 비율(90/10, 80/20 등)로 분배
- 헤더, 쿠키, 경로 등 조건별 라우팅
- 즉시 롤백 가능 (설정만 변경)

### VirtualService와 DestinationRule

```yaml
# DestinationRule: 버전(subset) 정의
spec:
  host: backend
  subsets:
    - name: v1
      labels:
        version: v1
    - name: v2
      labels:
        version: v2

# VirtualService: 라우팅 규칙
spec:
  route:
    - destination:
        host: backend
        subset: v1
      weight: 90
    - destination:
        host: backend
        subset: v2
      weight: 10
```

---

## 사전 준비: Backend v1, v2 배포

트래픽 분배를 테스트하기 위해 두 버전의 Backend 서비스를 배포합니다.

In [ ]:
# Backend v1 배포
!kubectl apply -f ../01-traffic-management/backend-v1.yaml

In [ ]:
# Backend v2 배포
!kubectl apply -f ../01-traffic-management/backend-v2.yaml

In [ ]:
# Backend 서비스 생성
!kubectl apply -f ../01-traffic-management/backend-service.yaml

In [ ]:
# Pod 시작 대기 (10초)
import time
print("Pod 시작 대기 중...")
time.sleep(10)

# 배포 확인
!kubectl get pods -n istio-demo -l app=backend

---

## DestinationRule 설정

서비스의 **버전(subset)**을 정의합니다. label selector로 v1, v2를 구분합니다.

In [ ]:
# DestinationRule 적용
!kubectl apply -f ../01-traffic-management/destination-rule.yaml

In [ ]:
# 생성된 DestinationRule 확인
!kubectl get destinationrule -n istio-demo

---

## 시나리오 1: Canary 배포 (90% v1, 10% v2)

### Canary 배포란?

새 버전을 소수의 트래픽에만 노출하여 위험을 최소화하는 배포 전략입니다.

```
사용자 트래픽 ─────┬────90%────► v1 (안정 버전)
                  └────10%────► v2 (새 버전)
```

**장점:**
- 문제 발생 시 10% 사용자만 영향
- 점진적으로 비율 증가 가능
- 롤백은 weight 변경만으로 즉시 가능

In [ ]:
# Canary VirtualService 적용
!kubectl apply -f ../01-traffic-management/virtual-service-canary.yaml

In [ ]:
# 20회 요청으로 트래픽 분배 테스트
v1_count = 0
v2_count = 0

print("20회 요청 중...")
for i in range(20):
    result = !kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://backend:8080 2>/dev/null
    response = result[0] if result else ""
    if "v1" in response:
        v1_count += 1
        print("1", end="", flush=True)
    elif "v2" in response:
        v2_count += 1
        print("2", end="", flush=True)
    else:
        print("?", end="", flush=True)

print(f"\n\n결과: v1={v1_count}회, v2={v2_count}회")
print(f"비율: v1={v1_count/20*100:.0f}%, v2={v2_count/20*100:.0f}%")
print("\n예상: v1≈90%, v2≈10%")

**결과 해석:**
- v1이 대부분(~90%)의 요청을 처리
- v2는 소수(~10%)만 처리
- 샘플 수가 적어 정확히 90/10이 아닐 수 있음

---

## 시나리오 2: 헤더 기반 라우팅

특정 HTTP 헤더가 있으면 특정 버전으로 라우팅합니다.

**사용 사례:**
- QA팀/Beta 테스터만 새 버전 접근
- A/B 테스트
- 특정 지역/언어 사용자 분리

In [ ]:
# 헤더 기반 라우팅 VirtualService 적용
!kubectl apply -f ../01-traffic-management/virtual-service-header.yaml

In [ ]:
# 테스트 1: 헤더 없이 요청 → v1으로 라우팅
print("테스트 1: 헤더 없이 요청 (→ v1 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://backend:8080

In [ ]:
# 테스트 2: x-version: v2 헤더로 요청 → v2로 라우팅
print("테스트 2: x-version: v2 헤더 (→ v2 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -H "x-version: v2" http://backend:8080

**결과 해석:**
- 헤더 없음 → v1 응답
- `x-version: v2` 헤더 → v2 응답
- 동일한 서비스인데 헤더에 따라 다른 버전으로 라우팅!

---

## 정리 (Cleanup)

테스트에 사용한 리소스를 삭제합니다.

In [ ]:
# VirtualService 삭제
!kubectl delete -f ../01-traffic-management/virtual-service-canary.yaml --ignore-not-found
!kubectl delete -f ../01-traffic-management/virtual-service-header.yaml --ignore-not-found

In [ ]:
# DestinationRule 삭제
!kubectl delete -f ../01-traffic-management/destination-rule.yaml --ignore-not-found

In [ ]:
# Backend 리소스 삭제
!kubectl delete -f ../01-traffic-management/backend-service.yaml --ignore-not-found
!kubectl delete -f ../01-traffic-management/backend-v1.yaml --ignore-not-found
!kubectl delete -f ../01-traffic-management/backend-v2.yaml --ignore-not-found
print("정리 완료!")

---

## 트래픽 관리 테스트 완료!

### 핵심 정리

| 기능 | 리소스 | 용도 |
|------|--------|------|
| 버전 정의 | DestinationRule | label로 v1, v2 구분 |
| 가중치 라우팅 | VirtualService | 90/10, 80/20 등 분배 |
| 조건부 라우팅 | VirtualService | 헤더, 경로, 쿠키 등 |

**Istio 트래픽 관리 장점:**
- 코드 수정 없이 배포 전략 변경
- 즉시 롤백 가능
- 세밀한 트래픽 제어

### 다음 단계

**[03-복원력.ipynb](./03-복원력.ipynb)** - 타임아웃, 재시도, 서킷 브레이커 등 복원력 기능을 학습합니다.